In [1]:
import json
import pandas as pd
import folium
from folium import plugins
import datetime

C:\Users\wangt\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\wangt\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
# with open('StationGeo.json','r',encoding='utf-8') as file:
#     statgeo = json.load(file)
# print(statgeo)
statdata = pd.read_csv('stations_data.csv',encoding='gbk')
statdata.head()

,Unnamed: 0,车站,拼音,路局,地区,经度x,纬度x,id,经度,纬度,省,市,区
0,60,合肥北城,HE FEI BEI CHENG,中国铁路上海局,安徽省 合肥市 长丰县,116.4698,33.3881,36990862.0,117.245500,32.064574,安徽省,合肥市,长丰县
1,61,合肥,HE FEI,中国铁路上海局,安徽省 合肥市 瑶海区,116.5339,33.2087,31900137.0,117.308645,31.886896,安徽省,合肥市,瑶海区
2,62,合肥南,HE FEI NAN,中国铁路上海局,安徽省 合肥市 包河区,116.5086,33.1204,8640433.0,117.283722,31.799444,安徽省,合肥市,包河区
3,172,安庆,AN QING,中国铁路上海局,安徽省 安庆市 宜秀区,116.2806,31.8599,43270283.0,117.059119,30.551045,安徽省,安庆市,宜秀区
4,173,安庆西,AN QING XI,中国铁路上海局,安徽省 安庆市 怀宁县,116.0771,31.9840,95364762.0,116.858651,30.673954,安徽省,安庆市,怀宁县


In [3]:
def correct(loc):
    lat = loc[0]
    lng = loc[1]
    deltalng = 0.0149 * lng - 2.5111
    deltalat = 0.0096 * lat +1.003
    loc = [lat-deltalat, lng-deltalng]
    return loc

In [4]:

statgeo = {}
for i in range(len(statdata)):
    lat = statdata['纬度'][i]
    lng = statdata['经度'][i]
    loc = [lat,lng]
#     loc = correct(loc)
    statgeo[statdata['车站'][i]]=loc
print(statgeo)    

{'合肥北城': [32.06457424, 117.2455], '合肥': [31.88689648, 117.3086449], '合肥南': [31.79944416, 117.2837219], '安庆': [30.55104496, 117.0591191], '安庆西': [30.6739536, 116.8586512], '蚌埠南': [32.9028488, 117.4353288], '蚌埠': [32.94404944, 117.3699181], '亳州': [33.84838368, 115.7914924], '亳州南': [33.79529824, 115.7909013], '巢湖东': [31.6192904, 117.9190129], '巢湖': [31.61622016, 117.8844358], '长临河': [31.69941376, 117.4768015], '滁州北': [32.3095992, 118.3211307], '池州': [30.620472, 117.5160084], '滁州': [32.19877344, 118.3134469], '砀山南': [34.38626992, 116.2965532], '东至': [30.17390064, 117.0224733], '肥东': [31.86263168, 117.4788702], '肥西': [31.71872656, 117.1366464], '阜阳': [32.91612016, 115.8632077], '阜阳西': [32.88422928, 115.7454882], '淮北': [33.968024, 116.7962944], '淮南': [32.63167728, 117.0129179], '淮南东': [32.64663232, 117.1440347], '淮南南': [32.54343264, 117.0344916], '黄山北': [29.81715856, 118.2666546], '黄山': [29.72396192, 118.3160082], '黄山西': [30.27848688, 118.0839186], '界首南': [33.22344128, 115.3181518], '绩溪北': [

In [5]:
data = pd.read_excel('火车乘坐记录.xlsx', sheet_name = 0)
data = data.fillna('nan')

In [6]:
a = data.iloc[0]
for i in a[5:]:
    if i == 'nan':
        break
    print(i)

珠海
前山
明珠
唐家湾
珠海北
南朗
中山
中山北
东升
小榄
南头
容桂
顺德学院
顺德


In [7]:
# 绘制起点与终点连线
# m = folium.Map(location=statgeo['上海'], zoom_start=5)
m = folium.Map(location=statgeo['上海'],
               zoom_start=5,
               control_scale=True,
               control=False,
               tiles=None
              )

folium.TileLayer(tiles='http://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png',
                 attr="&copy; <a href='https://stadiamaps.com/' target='_blank'>Stadia Maps</a> &copy; <a href='https://openmaptiles.org/' target='_blank'>OpenMapTiles</a> &copy; <a href='https://www.openstreetmap.org/copyright' target='_blank'>OpenStreetMap</a>&copy; <a href='https://stamen.com/' target='_blank'>Stamen Design</a>",
                 min_zoom=0,
                 max_zoom=19,
                 control=True,
                 show=True,
                 overlay=False,
                 name='ss'
                ).add_to(m)
polyline_set = []
point_set = []
heatmap_data = []

for i in range(len(data)):
    a = data.iloc[i]
    date = a.iloc[1]
    color = 'blue'
    if type(date) != pd._libs.tslibs.nattype.NaTType:
        dt = pd.to_datetime(date)
        if dt.year>2023:
            color = 'red'
    polyline = []
    
    for p in a[3:5]:
        if p == 'nan':
            continue
        if p not in statgeo:
            continue
        loc = statgeo[p]
        polyline.append(loc)
        heatmap_data.append(loc)
    
    polyline_set.append([polyline,color])
    point_set.append([loc,color])
    point_set.append([statgeo[a.iloc[5]],color])

for polyline,color in polyline_set:
    op = 0.3
    if color == 'blue':
        op = 0.1
    folium.PolyLine(
        locations = polyline,
        color=color,
        weight=6,
        opacity=op
    ).add_to(m)

for point,color in point_set:
    folium.Circle(
        location=point,
        radius=9000,   # 圆的半径
        popup=point,
        color=color, #'#FF1493',
        fill=True,
        fill_color='#FFD700'
    ).add_to(m)

m.save('map_line_rail.html')

In [17]:
#绘制路径
# m = folium.Map(location=statgeo['上海'], zoom_start=5)
m = folium.Map(location=statgeo['上海'],
               zoom_start=5,
               control_scale=True,
               control=False,
               tiles=None
              )

folium.TileLayer(tiles='http://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png',
                 attr="&copy; <a href='https://stadiamaps.com/' target='_blank'>Stadia Maps</a> &copy; <a href='https://openmaptiles.org/' target='_blank'>OpenMapTiles</a> &copy; <a href='https://www.openstreetmap.org/copyright' target='_blank'>OpenStreetMap</a>&copy; <a href='https://stamen.com/' target='_blank'>Stamen Design</a>",
                 min_zoom=0,
                 max_zoom=19,
                 control=True,
                 show=True,
                 overlay=False,
                 name='year'
                ).add_to(m)
polyline_set = []
point_set = []
heatmap_data = []

for i in range(len(data)):
    a = data.iloc[i]
    date = a.iloc[1]
    color = 'blue'
    if type(date) != pd._libs.tslibs.nattype.NaTType:
        dt = pd.to_datetime(date)
        if dt.year>2023:
            color = 'red'
    polyline = []
    
    for p in a[5:]:
        if p == 'nan':
            continue
        if p not in statgeo:
            continue
        loc = statgeo[p]
        name = p
        polyline.append(loc)
        heatmap_data.append(loc)
    
    polyline_set.append([polyline,color])
    point_set.append([loc,color,name])
    point_set.append([statgeo[a.iloc[5]],color,a.iloc[5]])

    
after = folium.FeatureGroup(name='since 2024')
before = folium.FeatureGroup(name='before 2024')
for polyline,color in polyline_set:
    op = 0.3
    group  = after
    if color == 'blue':
        op = 0.1
        group = before
    folium.PolyLine(
        locations = polyline,
        color=color,
        weight=6,
        opacity=op
    ).add_to(group)

for point,color,name in point_set:
    group = after
    if color == 'blue':
        group = before
    folium.Circle(
        location=point,
        radius=9000,   # 圆的半径
        popup=folium.Popup(name,max_width=10),
        color=color, #'#FF1493',
        fill=True,
        fill_color='#FFD700'
    ).add_to(group)

after.add_to(m)
before.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
m.save('map_rail.html')

In [9]:
#绘制热图
m = folium.Map(location=statgeo['上海'],
               zoom_start=5,
               control_scale=True,
               control=False,
               tiles=None
              )

folium.TileLayer(tiles='http://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png',
                 attr="&copy; <a href='https://stadiamaps.com/' target='_blank'>Stadia Maps</a> &copy; <a href='https://openmaptiles.org/' target='_blank'>OpenMapTiles</a> &copy; <a href='https://www.openstreetmap.org/copyright' target='_blank'>OpenStreetMap</a>&copy; <a href='https://stamen.com/' target='_blank'>Stamen Design</a>",
                 min_zoom=0,
                 max_zoom=19,
                 control=True,
                 show=True,
                 overlay=False,
                 name='ss'
                ).add_to(m)

plugins.HeatMap(heatmap_data).add_to(m)
m.save('heatmap.html')

In [10]:
from tqdm import tqdm
m = folium.Map(location=statgeo['上海'],
               zoom_start=5,
               control_scale=True,
               control=False,
               tiles=None
              )

folium.TileLayer(tiles='http://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png',
                 attr="&copy; <a href='https://stadiamaps.com/' target='_blank'>Stadia Maps</a> &copy; <a href='https://openmaptiles.org/' target='_blank'>OpenMapTiles</a> &copy; <a href='https://www.openstreetmap.org/copyright' target='_blank'>OpenStreetMap</a>&copy; <a href='https://stamen.com/' target='_blank'>Stamen Design</a>",
                 min_zoom=0,
                 max_zoom=19,
                 control=True,
                 show=True,
                 overlay=False,
                 name='ss'
                ).add_to(m)
point_set = []
for i in tqdm(range(len(statdata))):
    point_set.append(statgeo[statdata['车站'][i]])
    folium.Circle(
        location=statgeo[statdata['车站'][i]],
        radius=50,   # 圆的半径
        popup='Laurelhurst Park',
        color='#FF1493',
        fill=True,
        fill_color='#FFD700'
    ).add_to(m)
plugins.HeatMap(point_set,max_val=10).add_to(m)
m.save('map3.html')

100%|████████████████████████████████████████████████████████████████████████████| 3341/3341 [00:00<00:00, 6738.38it/s]
C:\Users\wangt\AppData\Local\Temp\ipykernel_47480\4133803472.py:29: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  plugins.HeatMap(point_set,max_val=10).add_to(m)


In [6]:
#统计数据计算
data = pd.read_excel('火车乘坐记录.xlsx', sheet_name = '乘坐列表')
#data = data.fillna(0)
data.head()

,序号,日期,车次,始发,终到,上车站,下车站,上车时间,下车时间,席位,座位,车型,车号,局属,时长,时长.1,价格,上车城市,下车城市,跨天
0,1,NaT,NaN,NaN,NaN,珠海,顺德,NaN,NaN,二等座,NaN,CRH1A,NaN,广铁,01:00:00,60.0,NaN,珠海市,佛山市,NaN
1,2,NaT,NaN,NaN,NaN,广州南,珠海,NaN,NaN,二等座,NaN,CRH1A,NaN,广铁,00:50:00,50.0,NaN,广州市,珠海市,NaN
2,3,NaT,NaN,NaN,NaN,广州南,北京西,NaN,NaN,二等座,NaN,NaN,NaN,广铁,11:00:00,660.0,NaN,广州市,北京市,NaN
3,4,NaT,Z201,NaN,NaN,北京西,广州,NaN,NaN,硬卧,NaN,YW25T,NaN,广铁,21:59:00,1319.0,NaN,北京市,广州市,1.0
4,5,2016-08-05,Z202,三亚,北京西,广州,北京西,08:50:00,06:49:00,硬卧,16车20号下,YW25T,NaN,广铁,21:59:00,1319.0,456.0,广州市,北京市,1.0


In [1]:

summary = '''
'''

#总乘坐次数
total = data.shape[0]
summary += f'''总乘坐次数：{total}
'''


#总乘坐时间
total_time = sum(data['时长.1'])
hour,minute = divmod(total_time,60)
summary += f'''总乘坐时长：{int(hour)}小时{int(minute)}分钟
'''

#乘坐时间最长和最短的车
timemax = data.loc[data['时长.1'].idxmax()]
hour,minute = divmod(timemax['时长.1'],60)
summary += f'''最长乘坐：{timemax['车次']}次从{timemax['上车站']}到{timemax['下车站']}共{int(hour)}小时{int(minute)}分钟
'''
timemin = data.loc[data['时长.1'].idxmin()]
hour,minute = divmod(timemin['时长.1'],60)
summary += f'''最短乘坐：{timemin['车次']}次从{timemin['上车站']}到{timemin['下车站']}共{int(hour)}小时{int(minute)}分钟
'''

#30分钟短途车
under30 = (data[data['时长.1']<=30].shape[0])
summary += f'''乘坐了30分钟内短途车{under30}次
'''

#跨天车
night = data[data['跨天']>=1].shape[0]
summary += f'''乘坐了跨天车{night}次
'''

#乘坐时间直方图
from matplotlib import pyplot as plt
import numpy as np
bins = np.arange(0,630,30)
print(bins)
plt.hist(data['时长.1'],bins = bins)
plt.show()

#最早出发与最晚出发
earlydep = data.loc[data['上车时间'][26:].idxmin()]
summary += f'''最早出发：{earlydep['车次']}次在{earlydep['上车时间']}于{earlydep['上车站']}出发前往{earlydep['下车站']}
'''
latedep = data.loc[data['上车时间'][26:].idxmax()]
summary += f'''最晚出发：{latedep['车次']}次在{latedep['上车时间']}于{latedep['上车站']}出发前往{latedep['下车站']}
'''

#最早到达与最晚到达
earlyarr = data.loc[data['下车时间'][26:].idxmin()]
summary += f'''最早到达：{earlyarr['车次']}次在{earlyarr['下车时间']}由{earlyarr['上车站']}到达{earlyarr['下车站']}
'''
latearr = data.loc[data['下车时间'][26:].idxmax()]
summary += f'''最晚出发：{latearr['车次']}次在{latearr['下车时间']}由{latearr['上车站']}到达{latearr['下车站']}
'''



print(summary)

NameError: name 'data' is not defined